In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime,timedelta

In [43]:
list1=["https://www.timesjobs.com/candidate/job-search.html?from=submit&searchType=personalizedSearch&txtLocation=Thiruvananthapuram&luceneResultSize=25&postWeek=60&pDate=Y&sequence=","1","&startPage=1"]
url="".join(list1)

In [29]:
def get_html_file(url): 
    source=requests.get(url).text
    soup=BeautifulSoup(source,'lxml')
    return soup

In [30]:
print(get_html_file(url).prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html><head>
<script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"2aa1afabd7",applicationID:"57254935"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o(e,[u.now()].concat(c(arguments)),t?null:this,n),t?void 0:this}}var o=e("handle"),a=e(6),c=e(7),f=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var d=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],p="api-",l=p+"ixn-";a(d,function(e,t){s[t]=i(p+t,!0,"api")}),s.addPageAction=i(p+"addPageAction",!0),s.setCurrentRouteName=i(p+"rou

In [31]:
soup=get_html_file(url)

In [141]:
def web_scrapping(content):
    list2=[]
    for i in content.stripped_strings:
        list2.append(i)
    job_role=list2[0]
    company_name=list2[1]
    #experience comes after card travel
    exp_req=list2[list2.index("card_travel")+1]
    if '₹' not in list2:
        salary="NA"
    else:
        salary=list2[list2.index('₹')+1]
    job_description=list2[list2.index('Job Description:')+1]
                 
    return company_name,job_role,salary,job_description,exp_req

In [130]:
def posted_date_scrapping(url):
    job_specific=get_html_file(url).find("div",class_="jd-header wht-shd-bx")
    list2=[]
    for i in job_specific.stripped_strings:
        list2.append(i)
    try:
        posted_date=pd.to_datetime(list2[-1].split("on")[-1]).date()
    except TypeError and ValueError:
        posted_date=datetime.today().date()
    return posted_date
    

### checking whether the scrapping works for the first page and the first job entry

In [142]:
content=soup.find('ul',class_="new-joblist").li

In [143]:
web_scrapping(content)

('BP Incorporate',
 'urgent opening placement officer anywhere in india',
 'Rs 0.60 - 20.60 Lacs p.a.',
 'internal project outsource Recruitment of suitable freelancer candidates like sales /counselor/telemarketing/recruiter/franchisee from lead and excel data provided by us . pro...',
 '1 - 6 yrs')

## job posted date is available only in the specific page for the job

In [101]:
job_specific_url=soup.find('ul',class_="new-joblist").li.h2.a.get("href")

In [131]:
posted_date_scrapping(job_specific_url)

datetime.date(2020, 9, 25)

### running the scrapper for all the job entries and pages

In [162]:
df1=pd.DataFrame(columns=["posted_date"])
df2=pd.DataFrame(columns=["Company Name","Job Role","Salary","Job Decription","Required Exp"])

In [163]:
web_list=["https://www.timesjobs.com/candidate/job-search.html?from=submit&searchType=personalizedSearch&txtLocation=Thiruvananthapuram&luceneResultSize=25&postWeek=60&pDate=Y&sequence=","1","&startPage=1"]
j=0

In [164]:

# 49 is the page number in the shine.com. The for loop is to run through all the pages. 
for i in range (1,49):
    web_list[1]=str(i)
    soup=get_html_file("".join(web_list))
    ### for looping through all the job posts in a single page
    for k in soup.find('ul',class_="new-joblist").find_all("li",class_="clearfix job-bx wht-shd-bx"):
        job_specific_url=k.h2.a.get("href")
        df1.loc[j]=posted_date_scrapping(job_specific_url)
        #the link for the specific job post
        if k!= None:
            df2.loc[j]=web_scrapping(k)
            j+=1
            print(j,end=" ")
        else:
            continue  

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [167]:
df=pd.concat([df2,df1],axis=1)

In [168]:
df

,Company Name,Job Role,Salary,Job Decription,Required Exp,posted_date
0,BP Incorporate,urgent opening for HR PARTNER,Rs 0.60 - 50.60 Lacs p.a.,"Freelancer/Franchisee HR PARTNER, desired cand...",13 - 18 yrs,2020-11-02
1,BP Incorporate,urgent opening placement officer anywhere in i...,Rs 0.60 - 20.60 Lacs p.a.,internal project outsource Recruitment of suit...,1 - 6 yrs,2020-11-02
2,BP Incorporate,urgent opening project manager placement services,Rs 0.60 - 20.60 Lacs p.a.,provide placement service to candidates by enr...,7 - 12 yrs,2020-11-02
3,BP Incorporate,"M.C.A., M.B.A., M.Sc., B.Tech.B.Tech(CSE, ECE,...",Rs 1.50 - 30.00 Lacs p.a.,Rolemechanical/electrical/electronics -product...,7 - 12 yrs,2020-11-02
4,Novature Tech,SAP Security,NA,Create and maintain user Roles and Authorizati...,4 - 6 yrs,2020-11-02
...,...,...,...,...,...,...
1195,Sat Prakash associates,"supervisor,""Building Operations Supervisor""",Rs 4.00 - 12.00 Lacs p.a.,Job DescriptionResponsibilities and Accountabi...,1 - 6 yrs,2020-09-10
1196,Sat Prakash associates,Sales & Marketing Manager,Rs 4.00 - 12.00 Lacs p.a.,Job DescriptionResponsibilities and Accountabi...,1 - 5 yrs,2020-09-10
1197,Sat Prakash associates,"Pharmacy Operations Manager,Pharmacy Assistant...",Rs 3.90 - 12.00 Lacs p.a.,Job DescriptionResponsibilities and Accountabi...,1 - 5 yrs,2020-09-10
1198,Doctor Door health care satffing and consultan...,RESIDENT MEDICAL OFFICER,NA,WANTED RESIDENT MEDICAL OFFICER Designation: R...,0 - 1 yrs,2020-09-07


In [169]:
df.to_csv("D:\\reach_technologies_internship\\timesjob.csv")